# Regression

## Linear Regression

In [1]:
using CmdStan
lr_stan_src = "
data {
    int<lower=0> N;
    vector[N] x;
    vector[N] y;
}
parameters {
    real alpha;
    real beta;
    real<lower=0> sigma;
}
model {
    y ~ normal(alpha + beta * x, sigma);
}
"
lr_model = Stanmodel(model = lr_stan_src)
lr_data = Dict("N" => 100, "x" => rand(100), "y" => rand(100))
lr_fit = stan(lr_model, lr_data)


File /media/weiya/Windows/Users/weiya/Documents/GitHub/techNotes/docs/stan/tmp/noname.stan will be updated.



Informational Message: The current Metropolis proposal is about to be rejected because of the following issue:
Exception: normal_lpdf: Scale parameter is 0, but must be > 0!  (in '/media/weiya/Windows/Users/weiya/Documents/GitHub/techNotes/docs/stan/tmp/noname.stan' at line 12)

If this warning occurs sporadically, such as for highly constrained variable types like covariance matrices, then the sampler is fine,
but if this warning occurs often then your model may be either severely ill-conditioned or misspecified.




Inference for Stan model: noname_model
4 chains: each with iter=(1000,1000,1000,1000); warmup=(0,0,0,0); thin=(1,1,1,1); 4000 iterations saved.

Warmup took (0.064, 0.065, 0.064, 0.069) seconds, 0.26 seconds total
Sampling took (0.091, 0.091, 0.088, 0.086) seconds, 0.36 seconds total

                 Mean     MCSE  StdDev     5%    50%   95%    N_Eff  N_Eff/s    R_hat
lp__               67  3.2e-02     1.2     65     68    69  1.4e+03  4.0e+03  1.0e+00
accept_stat__    0.93  1.5e-03   0.098   0.72   0.97   1.0  4.4e+03  1.2e+04  1.0e+00
stepsize__       0.32  2.2e-02   0.032   0.29   0.32  0.37  2.0e+00  5.6e+00  7.0e+13
treedepth__       3.0  1.6e-02    0.83    2.0    3.0   4.0  2.8e+03  7.8e+03  1.0e+00
n_leapfrog__       10  8.1e-02     4.8    3.0     11    15  3.4e+03  9.6e+03  1.0e+00
divergent__      0.00     -nan    0.00   0.00   0.00  0.00     -nan     -nan     -nan
energy__          -66  4.6e-02     1.7    -68    -66   -63  1.4e+03  4.0e+03  1.0e+00
alpha            0.45  1.

(0, [67.3108 0.884086 … -0.0901903 0.319224; 68.0402 0.998217 … 0.0820083 0.309652; … ; 64.8937 0.714864 … 0.0733968 0.27372; 67.94 1.0 … -0.0534246 0.313792]

[67.8875 0.992941 … -0.0267473 0.283699; 68.1533 0.992215 … 0.0788954 0.328354; … ; 68.4729 0.970527 … 0.0839659 0.296765; 66.3032 0.722181 … 0.0487446 0.316463]

[66.0761 0.761098 … 0.293095 0.31294; 65.3591 0.860735 … 0.258585 0.320708; … ; 68.3152 1.0 … 0.051309 0.315001; 67.5405 0.994056 … 0.184409 0.325249]

[67.5764 0.737473 … 0.092919 0.316847; 68.5774 0.940929 … 0.0665028 0.318573; … ; 67.2978 1.0 … 0.088559 0.273615; 68.6987 0.853552 … 0.0302446 0.313424], ["lp__", "accept_stat__", "stepsize__", "treedepth__", "n_leapfrog__", "divergent__", "energy__", "alpha", "beta", "sigma"])

## QR Reparameterization

In [6]:
lr_qr_stan_src = "
data {
    int<lower=0> N;
    int<lower=0> K;
    matrix[N, K] x;
    vector[N] y;
}
transformed data {
    matrix[N, K] Q_ast;
    matrix[K, K] R_ast;
    matrix[K, K] R_ast_inverse;
    Q_ast = qr_Q(x)[, 1:K] * sqrt(N-1);
    R_ast = qr_R(x)[1:K, ] / sqrt(N-1);
    R_ast_inverse = inverse(R_ast);
}
parameters {
    real alpha;
    vector[K] theta;
    real<lower=0> sigma;
}
model {
    y ~ normal(Q_ast * theta + alpha, sigma);
}
generated quantities {
    vector[K] beta;
    beta = R_ast_inverse * theta;
}
"
lr_qr_model = Stanmodel(model = lr_qr_stan_src)
x = rand(100, 4)
y = x[:,1] + x[:, 2] + x[:, 3] * 2 + x[:, 4] 
lr_qr_data = Dict("N" => 100, "K" => 4, "x" => x, "y" => y)
lr_fit = stan(lr_qr_model, lr_qr_data)

Informational Message: The current Metropolis proposal is about to be rejected because of the following issue:
Exception: normal_lpdf: Scale parameter is 0, but must be > 0!  (in '/home/weiya/cmdstan/tmp/noname.stan' at line 21)

If this warning occurs sporadically, such as for highly constrained variable types like covariance matrices, then the sampler is fine,
but if this warning occurs often then your model may be either severely ill-conditioned or misspecified.




Inference for Stan model: noname_model
4 chains: each with iter=(1000,1000,1000,1000); warmup=(0,0,0,0); thin=(1,1,1,1); 4000 iterations saved.

Warmup took (1.1, 1.1, 0.95, 1.2) seconds, 4.4 seconds total
Sampling took (2.4, 2.3, 2.0, 2.5) seconds, 9.2 seconds total

                    Mean     MCSE   StdDev        5%       50%       95%    N_Eff  N_Eff/s    R_hat
lp__             1.4e+03  6.3e+00  1.9e+01   1.4e+03   1.4e+03   1.4e+03  8.8e+00  9.6e-01  1.8e+00
accept_stat__    6.8e-01  1.2e-01  3.2e-01   3.7e-02   8.1e-01   1.0e+00  7.1e+00  7.6e-01  1.5e+00
stepsize__       2.1e-05  3.2e-06  4.5e-06   1.6e-05   2.1e-05   2.8e-05  2.0e+00  2.2e-01  2.4e+14
treedepth__      5.6e+00  5.4e-02  3.2e+00   2.0e+00   4.0e+00   1.0e+01  3.4e+03  3.7e+02  1.0e+00
n_leapfrog__     3.3e+02  8.3e+00  4.3e+02   3.0e+00   3.1e+01   1.0e+03  2.7e+03  2.9e+02  1.0e+00
divergent__      4.4e-02     -nan  2.1e-01   0.0e+00   0.0e+00   0.0e+00     -nan     -nan  1.1e+00
energy__        -1.4e+03  6.3e

(0, [1378.1 0.989084 … 2.0 1.0; 1376.41 0.965605 … 2.0 1.0; … ; 1440.19 0.21143 … 2.0 1.0; 1440.19 0.0212026 … 2.0 1.0]

[1380.41 0.97227 … 2.0 1.0; 1380.83 0.928805 … 2.0 1.0; … ; 1411.84 0.866959 … 2.0 1.0; 1415.28 1.0 … 2.0 1.0]

[1344.89 0.982476 … 2.0 1.0; 1344.66 0.990933 … 2.0 1.0; … ; 1410.63 0.357847 … 2.0 1.0; 1411.7 0.1556 … 2.0 1.0]

[1397.31 0.973037 … 2.0 1.0; 1397.8 0.893892 … 2.0 1.0; … ; 1450.17 0.952858 … 2.0 1.0; 1449.52 0.941807 … 2.0 1.0], ["lp__", "accept_stat__", "stepsize__", "treedepth__", "n_leapfrog__", "divergent__", "energy__", "alpha", "theta.1", "theta.2", "theta.3", "theta.4", "sigma", "beta.1", "beta.2", "beta.3", "beta.4"])

# Gaussian Processes

Gaussian processes are continuous stochastic processes and thus may be interpreted as providing a probability distribution over functions.

Gaussian process is parameterized by a mean function and covariance function. 

## Gaussian Processes Regression

data: $x_1,\ldots,x_N\in \mathbb{R}^D$ paired with $y_1,\ldots,y_N\in\mathbb{R}$.

$$
y\sim \mathrm{MvNormal}(m(x), K(x\mid\theta))\,,
$$

where $m:\mathbb{R}\rightarrow \mathbb{R}^N$ and $K:\mathbb{R}^{N\times D}\rightarrow \mathbb{R}^{N\times N}$ must produce a positive-definite matrix for any input $x$.

A popular covariance function

$$
K(x\mid \alpha,\rho,\sigma)_{i,j}=\alpha^2\exp\left(-\frac{1}{2\rho^2}\sum_{d=1}^D(x_{i,d}-x_{j,d})^2\right)+\delta_{i,j}\sigma^2\,,
$$

where $\alpha,\rho,\sigma$ are hyperparameters defining the covariance function and where $\delta_{i,j}$ is the Kronecker delta function with value if $i=j$ and value $0$ otherwise.

Note that the kernel is obtained through a convolution of two independent Gaussian processes, $f_1$ and $f_2$, with kernels

$$
K_1(x\mid\alpha,\rho)_{i,j} = \alpha^2\exp\left(-\frac{1}{2\rho^2}\sum_{d=1}^D(x_{i,d}-x_{i,d})^2\right)
$$

and

$$
K_2(x\mid\sigma)_{i,j}=\delta_{i,j}\sigma^2\,.
$$

- $\sigma^2$:
- $\rho$: length-scale
- $\alpha$: marginal standard deviation

## Simulating from a Gaussian Process


In [8]:
gp_stan_src = "
data {
    int<lower=1> N;
    real x[N];
}
transformed data {
    matrix[N, N] K = cov_exp_quad(x, 1.0, 1.0);
    vector[N] mu = rep_vector(0, N);
    /*
    for (i in 1:(N-1)){
        K[i, i] = 1 + 0.1;
        for (j in (i+1):N)
        {
            K[i, j] = exp(-0.5 * square(x[i] - x[j]));
            K[j, i] = K[i, j];
        }
    }
    K[N, N] = 1 + 0.1;
    */
    for (n in 1:N)
    {
        K[n, n] = K[n, n] + 0.1;
    }
}
parameters {
    vector[N] y;
}
model {
    y ~ multi_normal(mu, K);
}
"
gp_model = Stanmodel(model = gp_stan_src)
gp_data = Dict("N" => 10, "x" => rand(10))
stan(gp_model, gp_data)


File /home/weiya/cmdstan/tmp/noname.stan will be updated.


Inference for Stan model: noname_model
4 chains: each with iter=(1000,1000,1000,1000); warmup=(0,0,0,0); thin=(1,1,1,1); 4000 iterations saved.

Warmup took (0.070, 0.067, 0.070, 0.065) seconds, 0.27 seconds total
Sampling took (0.079, 0.069, 0.077, 0.086) seconds, 0.31 seconds total

                    Mean     MCSE  StdDev    5%       50%   95%    N_Eff  N_Eff/s    R_hat
lp__            -5.0e+00  6.6e-02     2.2  -9.1  -4.7e+00  -2.0  1.2e+03  3.7e+03  1.0e+00
accept_stat__    9.0e-01  1.6e-03    0.10  0.69   9.3e-01   1.0  4.5e+03  1.5e+04  1.0e+00
stepsize__       2.3e-01     -nan   0.010  0.21   2.3e-01  0.24     -nan     -nan  3.2e+13
treedepth__      3.3e+00  1.2e-02    0.69   2.0   3.0e+00   5.0  3.6e+03  1.1e+04  1.0e+00
n_leapfrog__     1.5e+01  2.5e-01      10   3.0   1.5e+01    35  1.7e+03  5.5e+03  1.0e+00
divergent__      0.0e+00     -nan    0.00  0.00   0.0e+00  0.00     -nan     -nan     -nan
energy__        

(0, [-4.40928 0.905573 … 0.50035 1.34142; -1.9529 0.98648 … 0.896582 0.349286; … ; -2.51878 0.823785 … -1.07043 -1.45072; -3.97118 0.584175 … -1.39329 -1.6131]

[-4.56643 0.887791 … -0.00929212 -0.0172257; -3.57976 0.978547 … 0.000471542 -0.0779251; … ; -7.29975 0.996586 … 2.56505 2.68298; -8.46521 0.9326 … 2.30839 2.68763]

[-4.3954 0.990607 … 0.642963 0.267167; -7.10518 0.90519 … 0.657146 0.161806; … ; -4.48538 0.993403 … 0.156624 0.0235797; -4.7357 0.756217 … 0.138927 -0.228088]

[-11.2698 0.896925 … 1.60593 1.71114; -9.99977 0.921981 … 1.16928 1.22872; … ; -2.22473 0.991267 … 1.04683 0.947586; -2.57243 0.965165 … 1.14268 1.4535], ["lp__", "accept_stat__", "stepsize__", "treedepth__", "n_leapfrog__", "divergent__", "energy__", "y.1", "y.2", "y.3", "y.4", "y.5", "y.6", "y.7", "y.8", "y.9", "y.10"])

In [9]:
gp_chol_stan_src ="
data {
    int<lower=1> N;
    real x[N];
}
transformed data {
    matrix[N, N] K = cov_exp_quad(x, 1.0, 1.0);
    matrix[N, N] L;
    vector[N] mu = rep_vector(0, N);
    for (n in 1:N){
        K[n, n] = K[n, n] + 0.1;
    }
    L = cholesky_decompose(K);
}
parameters {
    vector[N] eta;
}
model {
    eta ~ normal(0, 1);
}
generated quantities {
    vector[N] y;
    y = mu + L * eta;
}
"
gp_chol_model = Stanmodel(model = gp_chol_stan_src)
stan(gp_chol_model, gp_data)


File /home/weiya/cmdstan/tmp/noname.stan will be updated.


Inference for Stan model: noname_model
4 chains: each with iter=(1000,1000,1000,1000); warmup=(0,0,0,0); thin=(1,1,1,1); 4000 iterations saved.

Warmup took (0.018, 0.019, 0.023, 0.020) seconds, 0.081 seconds total
Sampling took (0.030, 0.037, 0.030, 0.067) seconds, 0.16 seconds total

                    Mean     MCSE  StdDev    5%       50%   95%    N_Eff  N_Eff/s    R_hat
lp__            -5.0e+00  5.3e-02     2.2  -9.1  -4.7e+00  -2.0  1.7e+03  1.1e+04  1.0e+00
accept_stat__    8.8e-01  1.2e-02    0.12  0.63   9.2e-01   1.0  1.0e+02  6.1e+02  1.0e+00
stepsize__       7.2e-01  4.3e-02   0.061  0.64   7.7e-01  0.79  2.0e+00  1.2e+01  5.7e+13
treedepth__      3.1e+00  1.7e-01    0.90   2.0   3.0e+00   5.0  2.6e+01  1.6e+02  1.1e+00
n_leapfrog__     1.4e+01  6.2e+00      29   3.0   7.0e+00    47  2.2e+01  1.3e+02  1.1e+00
divergent__      0.0e+00     -nan    0.00  0.00   0.0e+00  0.00     -nan     -nan     -nan
energy__       

(0, [-13.327 0.979241 … 0.109707 1.06823; -9.32145 0.994517 … -0.294982 -0.758803; … ; -8.62671 0.722279 … 0.588486 1.65773; -5.92529 1.0 … -1.15606 -1.61947]

[-4.88609 0.872102 … 1.20348 0.682431; -3.75551 0.696431 … -0.597714 -0.963524; … ; -11.4144 0.852733 … -1.52255 -0.334438; -2.67301 1.0 … -0.879083 -0.62328]

[-3.82347 0.997044 … 0.121559 -0.835292; -3.65942 0.772413 … 0.19914 0.131641; … ; -5.22656 0.920915 … -0.696994 -1.08691; -5.74705 0.935558 … 1.65489 1.94062]

[-4.24717 0.851841 … 0.739018 0.572328; -5.55245 0.753355 … -0.423088 -0.528525; … ; -7.75393 0.995804 … 1.74508 1.76892; -4.76191 0.981522 … 0.926099 1.08731], ["lp__", "accept_stat__", "stepsize__", "treedepth__", "n_leapfrog__", "divergent__", "energy__", "eta.1", "eta.2", "eta.3"  …  "y.1", "y.2", "y.3", "y.4", "y.5", "y.6", "y.7", "y.8", "y.9", "y.10"])

## Fitting a Gaussian Process



In [15]:
fitgp_stan_src ="
data {
    int<lower=1> N;
    real x[N];
    vector[N] y;
}
transformed data {
    vector[N] mu = rep_vector(0, N);
}
parameters {
    real<lower=0> rho;
    real<lower=0> alpha;
    real<lower=0> sigma;
}
model {
    matrix[N, N] L_K;
    matrix[N, N] K = cov_exp_quad(x, alpha, rho);
    real sq_sigma = square(sigma);
    
    // diagonal elements
    for (n in 1:N){
        K[n, n] = K[n, n] + sq_sigma;
    }
    
    L_K = cholesky_decompose(K);
    
    rho ~ inv_gamma(5, 5);
    alpha ~ normal(0, 1);
    sigma ~ normal(0, 1);
    y ~ multi_normal_cholesky(mu, L_K);
}
"
fitgp_model = Stanmodel(model = fitgp_stan_src)
fitgp_data = Dict("N"=> 10, "x"=> rand(10), "y"=>rand(10))
stan(fitgp_model, fitgp_data)


File /home/weiya/cmdstan/tmp/noname.stan will be updated.



Informational Message: The current Metropolis proposal is about to be rejected because of the following issue:
Exception: cholesky_decompose: Matrix m is not positive definite  (in '/home/weiya/cmdstan/tmp/noname.stan' at line 24)

If this warning occurs sporadically, such as for highly constrained variable types like covariance matrices, then the sampler is fine,
but if this warning occurs often then your model may be either severely ill-conditioned or misspecified.

Informational Message: The current Metropolis proposal is about to be rejected because of the following issue:
Exception: cholesky_decompose: Matrix m is not positive definite  (in '/home/weiya/cmdstan/tmp/noname.stan' at line 24)

If this warning occurs sporadically, such as for highly constrained variable types like covariance matrices, then the sampler is fine,
but if this warning occurs often then your model may be either severely ill-conditioned or misspecified.




Inference for Stan model: noname_model
4 chains: each with iter=(1000,1000,1000,1000); warmup=(0,0,0,0); thin=(1,1,1,1); 4000 iterations saved.

Warmup took (0.065, 0.065, 0.063, 0.065) seconds, 0.26 seconds total
Sampling took (0.070, 0.072, 0.068, 0.073) seconds, 0.28 seconds total

                Mean     MCSE  StdDev    5%   50%    95%    N_Eff  N_Eff/s    R_hat
lp__            -1.9  2.9e-02     1.2  -4.4  -1.6  -0.60  1.8e+03  6.4e+03  1.0e+00
accept_stat__   0.93  1.5e-03   0.096  0.74  0.96    1.0  4.4e+03  1.5e+04  1.0e+00
stepsize__      0.69  1.0e-02   0.014  0.66  0.70   0.70  2.0e+00  7.1e+00  1.1e+13
treedepth__      2.4  9.3e-03    0.56   2.0   2.0    3.0  3.7e+03  1.3e+04  1.0e+00
n_leapfrog__     5.3  3.6e-02     2.3   3.0   7.0    7.0  3.8e+03  1.4e+04  1.0e+00
divergent__     0.00     -nan    0.00  0.00  0.00   0.00     -nan     -nan     -nan
energy__         3.4  4.4e-02     1.7   1.2   3.0    6.7  1.6e+03  5.6e+03  1.0e+00
rho              1.3  1.3e-02    0.68  0.

(0, [-1.3471 1.0 … 0.457106 0.22393; -1.02156 0.931032 … 1.04016 0.231725; … ; -5.05079 0.686575 … 0.636851 0.640848; -7.68519 0.851448 … 1.23046 0.146177]

[-2.36374 0.989214 … 0.423633 0.26308; -1.33213 0.956551 … 1.25729 0.232365; … ; -1.43081 0.992552 … 0.529169 0.210934; -0.803672 0.709535 … 0.663358 0.278871]

[-2.18288 0.836305 … 1.24265 0.232148; -1.37366 0.982242 … 0.40109 0.275472; … ; -4.78596 0.779635 … 0.72266 0.387731; -2.56657 0.866996 … 0.246019 0.21628]

[-0.755367 0.989393 … 0.477886 0.264234; -1.48958 0.57356 … 0.865481 0.19301; … ; -1.17014 0.952058 … 0.413781 0.215326; -2.13268 0.82529 … 0.62786 0.438014], ["lp__", "accept_stat__", "stepsize__", "treedepth__", "n_leapfrog__", "divergent__", "energy__", "rho", "alpha", "sigma"])